# SmartProjectionRigFactor

`SmartProjectionRigFactor<CAMERA>` is a generalization of `SmartProjectionPoseFactor` designed for multi-camera systems (rigs).
Like other smart factors, it implicitly represents a 3D point landmark observed by multiple cameras.

Key differences/features:
- **Multi-Camera Rig:** Assumes a fixed rig configuration, where multiple cameras (`CAMERA` instances, which include fixed intrinsics and fixed extrinsics *relative to the rig's body frame*) are defined.
- **Pose Variables:** Connects `Pose3` variables representing the pose of the **rig's body frame** in the world.
- **Multiple Observations per Pose:** Allows multiple measurements associated with the *same* body pose key, but originating from different cameras within the rig.
- **Camera Indexing:** Each measurement must be associated with both a body pose key and a `cameraId` (index) specifying which camera in the rig took the measurement.
- **Fixed Calibration/Extrinsics:** The intrinsics and relative extrinsics of the cameras within the rig are assumed fixed.
- **`CAMERA` Template:** Can be templated on various camera models (e.g., `PinholeCamera`, `SphericalCamera`), provided they adhere to the expected GTSAM camera concepts.
- **`Values` Requirement:** Requires `Pose3` objects (representing the body frame) in the `Values` container.
- **Configuration:** Behavior controlled by `SmartProjectionParams`. **Note:** Currently (as of header comment), only supports `HESSIAN` linearization mode and `ZERO_ON_DEGENERACY` mode.

**Use Case:** Ideal for visual SLAM with a calibrated multi-camera rig (e.g., stereo rig, multi-fisheye system) where only the rig's pose is optimized.

If you are using the factor, please cite:
> **L. Carlone, Z. Kira, C. Beall, V. Indelman, F. Dellaert**, "Eliminating conditionally independent sets in factor graphs: a unifying perspective based on smart factors", Int. Conf. on Robotics and Automation (ICRA), 2014.


<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/slam/doc/SmartProjectionRigFactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import google.colab
    %pip install --quiet gtsam-develop
except ImportError:
    pass  # Not running on Colab, do nothing

In [1]:
import gtsam
import numpy as np
from gtsam import (
    Values,
    Point2,
    Point3,
    Pose3,
    Rot3,
    NonlinearFactorGraph,
    SmartProjectionParams,
    SmartProjectionRigFactorPinholePoseCal3_S2,
    PinholePoseCal3_S2,
    CameraSetPinholePoseCal3_S2,
    Cal3_S2,
)
from gtsam.symbol_shorthand import X  # Key for Pose3 variable (Body Pose)

## Creating the Rig and Factor

1. Define the camera rig configuration: Create a `CameraSet` containing the `CAMERA` objects (with fixed intrinsics and rig-relative extrinsics).
2. Create the `SmartProjectionRigFactor` with noise, the rig, and parameters.
3. Add measurements, specifying the 2D point, the corresponding **body pose key**, and the **camera ID** within the rig.

In [3]:
# 1. Define the Camera Rig
K = Cal3_S2(500, 500, 0, 320, 240)
# Camera 0: Forward facing, slightly offset
body_P_cam0 = Pose3(Rot3.Ypr(0, 0, 0), Point3(0.1, 0, 0))
cam0 = PinholePoseCal3_S2(body_P_cam0, K)
# Camera 1: Stereo camera, right of cam0
body_P_cam1 = Pose3(Rot3.Ypr(0, 0, 0), Point3(0.1, -0.1, 0)) # Baseline 0.1m
cam1 = PinholePoseCal3_S2(body_P_cam1, K)

rig_cameras = gtsam.CameraSetPinholePoseCal3_S2()
rig_cameras.push_back(cam0)
rig_cameras.push_back(cam1)

# 2. Create the Factor
noise_model = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)
# Ensure parameters are compatible (HESSIAN, ZERO_ON_DEGENERACY)
smart_params = SmartProjectionParams(linMode=gtsam.LinearizationMode.HESSIAN,
                                     degMode=gtsam.DegeneracyMode.ZERO_ON_DEGENERACY)

# Factor type includes the Camera type
smart_factor = SmartProjectionRigFactorPinholePoseCal3_S2(noise_model, rig_cameras, smart_params)

# 3. Add measurements
# Observation from Body Pose X(0), Camera 0
smart_factor.add(Point2(300, 200), X(0), 0)
# Observation from Body Pose X(0), Camera 1 (stereo pair?)
smart_factor.add(Point2(250, 201), X(0), 1)
# Observation from Body Pose X(1), Camera 0
smart_factor.add(Point2(310, 210), X(1), 0)
# Observation from Body Pose X(1), Camera 1
smart_factor.add(Point2(260, 211), X(1), 1)

print(f"Smart factor involves {smart_factor.size()} measurements from {len(smart_factor.keys())} unique poses.")
smart_factor.print("SmartFactorRig: ")

Smart factor involves 2 measurements from 2 unique poses.
SmartFactorRig: SmartProjectionRigFactor: 
 -- Measurement nr 0
key: x0
cameraId: 0
camera in rig:
.pose R: [
	1, 0, 0;
	0, 1, 0;
	-0, 0, 1
]
t: 0.1   0   0
camera in rig:
.calibration[
	500, 0, 320;
	0, 500, 240;
	0, 0, 1
]
-- Measurement nr 1
key: x0
cameraId: 1
camera in rig:
.pose R: [
	1, 0, 0;
	0, 1, 0;
	-0, 0, 1
]
t:  0.1 -0.1    0
camera in rig:
.calibration[
	500, 0, 320;
	0, 500, 240;
	0, 0, 1
]
-- Measurement nr 2
key: x1
cameraId: 0
camera in rig:
.pose R: [
	1, 0, 0;
	0, 1, 0;
	-0, 0, 1
]
t: 0.1   0   0
camera in rig:
.calibration[
	500, 0, 320;
	0, 500, 240;
	0, 0, 1
]
-- Measurement nr 3
key: x1
cameraId: 1
camera in rig:
.pose R: [
	1, 0, 0;
	0, 1, 0;
	-0, 0, 1
]
t:  0.1 -0.1    0
camera in rig:
.calibration[
	500, 0, 320;
	0, 500, 240;
	0, 0, 1
]
SmartProjectionFactor
linearizationMode: 0
triangulationParameters:
rankTolerance = 1
enableEPI = 0
landmarkDistanceThreshold = -1
dynamicOutlierRejectionThreshold = -1

## Evaluating the Error

The `.error(values)` method uses the `Pose3` objects (body poses) from `values` and the fixed rig configuration to triangulate the point and compute the error.

In [4]:
# Create Values containing Body Pose3 objects
values = Values()
pose0 = Pose3(Rot3.Ypr(0.0, 0.0, 0.0), Point3(0, 0, 0))
pose1 = Pose3(Rot3.Ypr(0.1, 0.0, 0.0), Point3(0.5, 0, 0))
values.insert(X(0), pose0)
values.insert(X(1), pose1)

# Triangulate first to see the implicit point
# The 'cameras' method internally combines body poses with rig extrinsics
point_result = smart_factor.point(values)
print(f"Triangulated point result:\n{point_result}")

if point_result.valid():
   # Calculate error
   total_error = smart_factor.error(values)
   print(f"\nTotal reprojection error (0.5 * sum(err^2/sigma^2)): {total_error:.4f}")
else:
   print("\nTriangulation failed, error calculation depends on degeneracyMode.")
   # Since mode is ZERO_ON_DEGENERACY, error should be 0
   total_error = smart_factor.error(values)
   print(f"Error when degenerate: {total_error}")

Triangulated point result:

Triangulation failed, error calculation depends on degeneracyMode.
Error when degenerate: 0.0
